In [ ]:
!pip install langchain_objectbox
!pip install langchainhub
!pip install langchain_openai
!pip install langchain_core

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [ ]:
loader = WebBaseLoader([
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models', 
    'https://lmsys.org/blog/2023-03-30-vicuna/'
])
data = loader.load()
data

# 2. Convert data to Vector Database


In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [ ]:
documents

In [ ]:

vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

# 3. Make a RAG pipeline


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [ ]:
question = "How big is stableLM?"
result = qa_chain({"query": question })

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])